<a href="https://colab.research.google.com/github/Anisha-bhola/lab/blob/main/train%20dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas plotly dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv("train.csv")

# Convert 'Order Date' to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True)

# Extract Year and Month
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month_name()

# Display quick summary
print(df[['Order Date', 'Sales', 'Region', 'Year', 'Month']].head())

  Order Date     Sales Region  Year     Month
0 2017-11-08  261.9600  South  2017  November
1 2017-11-08  731.9400  South  2017  November
2 2017-06-12   14.6200   West  2017      June
3 2016-10-11  957.5775  South  2016   October
4 2016-10-11   22.3680  South  2016   October


In [3]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Year', 'Month'],
      dtype='object')

In [4]:
#Monthly sales trend
import plotly.express as px

monthly_sales = df.groupby(['Year', 'Month'])['Sales'].sum().reset_index()

# To ensure correct month order
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_sales['Month'] = pd.Categorical(monthly_sales['Month'], categories=month_order, ordered=True)
monthly_sales = monthly_sales.sort_values(by=['Year', 'Month'])

fig = px.line(monthly_sales, x='Month', y='Sales', color='Year',
              title='Monthly Sales Over the Years')
fig.show()

In [5]:
#Sales by region

region_sales = df.groupby('Region')['Sales'].sum().reset_index()

fig = px.bar(region_sales, x='Region', y='Sales',
             title='Sales by Region', color='Region')
fig.show()

In [6]:
#Sales by category

category_sales = df.groupby('Category')['Sales'].sum().reset_index()

fig = px.pie(category_sales, names='Category', values='Sales',
             title='Sales Distribution by Category')
fig.show()

In [7]:
#DASHBOARD

In [8]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Initialize Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Sales Dashboard - Train Dataset"),

    html.Label("Select Region:"),
    dcc.Dropdown(
        id='region-dropdown',
        options=[{'label': r, 'value': r} for r in df['Region'].unique()],
        value=df['Region'].unique()[0],
        multi=False
    ),

    dcc.Graph(id='monthly-sales-graph'),
    dcc.Graph(id='category-sales-graph')
])

# Callbacks
@app.callback(
    [Output('monthly-sales-graph', 'figure'),
     Output('category-sales-graph', 'figure')],
    [Input('region-dropdown', 'value')]
)
def update_graphs(region):
    filtered_df = df[df['Region'] == region]

    # Monthly Sales
    ms = filtered_df.groupby(['Year', 'Month'])['Sales'].sum().reset_index()
    ms['Month'] = pd.Categorical(ms['Month'], categories=month_order, ordered=True)
    ms = ms.sort_values(by=['Year', 'Month'])
    fig1 = px.line(ms, x='Month', y='Sales', color='Year', title=f'Monthly Sales in {region}')

    # Category Sales
    cs = filtered_df.groupby('Category')['Sales'].sum().reset_index()
    fig2 = px.bar(cs, x='Category', y='Sales', color='Category', title=f'Sales by Category in {region}')

    return fig1, fig2

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>

In [9]:
total_sales = df['Sales'].sum()
top_region = df.groupby('Region')['Sales'].sum().idxmax()
top_category = df.groupby('Category')['Sales'].sum().idxmax()

print(f"Total Sales: ${total_sales:,.2f}")
print(f"Top Region: {top_region}")
print(f"Top Category: {top_category}")


Total Sales: $2,261,536.78
Top Region: West
Top Category: Technology
